In [1]:
import xgboost as xgb
import alpaca_trade_api as tradeapi
from finta import TA
import pickle

# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv

In [2]:
papertrading_model = xgb.XGBClassifier()
papertrading_model.load_model('xgb_clf.bst')
scaler = pickle.load(open('scaler_model.pkl','rb'))



/Users/adriencaudron/miniforge3/envs/dev310/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 1.2.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:
feats = ['HMA_5', 'RSI_5', 'ATR_14', 'RSI_14', 'RSI_150', 'cci']

In [4]:
# Load environment variables
load_dotenv()

# Set Alpaca API key and secret
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')

In [5]:
alpaca_api_key

'PK2RJWWRGDVPCFAT43MM'

In [7]:
alpaca_api = tradeapi.REST(alpaca_api_key,alpaca_secret_key, 'https://paper-api.alpaca.markets')

In [8]:
data_trade = alpaca_api.get_crypto_bars(['BTC/USDT'], tradeapi.TimeFrame.Hour, "2023-01-01").df

In [9]:
for time in [5,14,150]:
    data_trade['RSI_'+str(time)] = TA.RSI(data_trade, time)
data_trade['ATR_14'] = TA.ATR(data_trade, 14)
data_trade['HMA_5'] = TA.HMA(data_trade, 5)
data_trade['cci'] = TA.CCI(data_trade)

In [10]:
data_trade.dtypes

close          float64
high           float64
low            float64
trade_count      int64
open           float64
volume         float64
vwap           float64
symbol          object
RSI_5          float64
RSI_14         float64
RSI_150        float64
ATR_14         float64
HMA_5          float64
cci            float64
dtype: object

In [11]:
# transform preprocessed data to numpy array
data_to_predict = scaler.transform(data_trade[feats].dropna())

In [12]:
data_to_predict[-1]

array([92.82887896, -0.32046975, 40.00417649, -0.42515975,  1.25311037,
       -0.67546317])

In [13]:
# make predictions using the model
predictions = papertrading_model.predict(data_to_predict)

In [14]:
BUY = predictions[-1]
display(BUY)

1

In [15]:
# Get the current price of BTC in USD
eth_usd_price = alpaca_api.get_latest_crypto_orderbook(['ETH/USD'])['ETH/USD'].asks[0].p

# Calculate the quantity of BTC to buy
usd_balance = float(alpaca_api.get_account().cash)
quantity_to_buy = usd_balance / eth_usd_price * 0.1

In [16]:
if BUY: 

## TEST to place a buy order for 500 USD worth of ETH using Alpaca API in Python:

    # Define order parameters
    symbol = 'ETHUSD'
    order_type = 'limit'
    limit_price = eth_usd_price # limit price for the buy order
    time_in_force = 'gtc'
    qty = quantity_to_buy # quantity of ETH to trade
    side = 'buy'

    # Place buy order
    try:
        order = alpaca_api.submit_order(
            symbol=symbol,
            qty=qty,
            side=side,
            type=order_type,
            time_in_force=time_in_force,
            limit_price=limit_price +0.0001
        )
        print(f"Buy order for {qty} {symbol} at {limit_price} submitted successfully.")

    except Exception as e:
        print("Error submitting buy order: ", e)

else:
    # Define order parameters
    symbol = 'ETHUSD'
    order_type = 'limit'
    limit_price = eth_usd_price  # limit price for the sell order
    time_in_force = 'gtc'
    qty = quantity_to_buy # quantity of ETH to sell
    qty = float(alpaca_api.get_position(symbol).qty) # quantity of ETH to sell (same as the current position)
    side = 'sell'

    # Place sell order
    try:
        order = alpaca_api.submit_order(
            symbol=symbol,
            qty=qty,
            side=side,
            type=order_type,
            time_in_force=time_in_force,
            limit_price=limit_price -0.0001
        )
        print(f"Sell order for {qty} {symbol} at {limit_price} submitted successfully.")

    except Exception as e:
        print("Error submitting sell order: ", e)



Buy order for 4.410182541971596 ETHUSD at 1662.3 submitted successfully.


In [17]:
eth_usd_price

1662.3

In [18]:
quantity_to_buy

4.410182541971596

In [19]:
def trade_in_alpaca(quantity, tries=0):
    if papertrading_model.predict(data_to_predict)[-1]==1 and papertrading_model.predict(data_to_predict)[-2]==0:
        try:
            alpaca_api.submit_order(symbol='BTC/USDT',qty=0.04,side="buy",type='market', time_in_force='ioc')
        except Exception as e:
            if 'insufficient balance' in str(e) and tries<=10:
                print(quantity, tries)
                trade_in_alpaca(quantity=0.9*quantity, tries=tries+1)
            else:
                print(e)

In [20]:
trade_in_alpaca(quantity=quantity_to_buy)

In [21]:
alpaca_api.list_positions()

[Position({   'asset_class': 'crypto',
     'asset_id': '35f33a69-f5d6-4dc9-b158-4485e5e92e4b',
     'asset_marginable': False,
     'avg_entry_price': '1664.1606557645993435',
     'change_today': '-0.0308052172289888',
     'cost_basis': '34490.009209651345586',
     'current_price': '1661.51',
     'exchange': 'FTXU',
     'lastday_price': '1714.32',
     'market_value': '34435.07392355624',
     'qty': '20.725168024',
     'qty_available': '20.725168024',
     'side': 'long',
     'symbol': 'ETHUSD',
     'unrealized_intraday_pl': '-54.935286095105585077592244',
     'unrealized_intraday_plpc': '-0.0015927883857953',
     'unrealized_pl': '-54.935286095105586',
     'unrealized_plpc': '-0.0015927883857953'})]

In [22]:
import xgboost as xgb
import alpaca_trade_api as tradeapi
from finta import TA
import pickle
import os
import time
from dotenv import load_dotenv
import pandas as pd

# Load environment variables
load_dotenv()

# Set Alpaca API key and secret
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')

# Import saved XGB model
papertrading_model = xgb.XGBClassifier()
papertrading_model.load_model('xgb_clf.bst')

# Load scaler model
scaler = pickle.load(open('scaler_model.pkl', 'rb'))

# Define list of features
feats = ['HMA_5', 'RSI_5', 'ATR_14', 'RSI_14', 'RSI_150', 'cci']

# Initialize Alpaca API
alpaca_api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, 'https://paper-api.alpaca.markets')


# Get latest data on 1HR timeframe
data_trade = alpaca_api.get_crypto_bars(['BTC/USDT'], tradeapi.TimeFrame.Hour, "2023-01-01").df

# Compute the features needed for the prediction
for time in [5, 14, 150]:
    data_trade['RSI_'+str(time)] = TA.RSI(data_trade, time)
data_trade['ATR_14'] = TA.ATR(data_trade, 14)
data_trade['HMA_5'] = TA.HMA(data_trade, 5)
data_trade['cci'] = TA.CCI(data_trade)

# Transform preprocessed data to numpy array
data_to_predict = scaler.transform(data_trade[feats].dropna())

# Make predictions using the model
predictions = papertrading_model.predict(data_to_predict)

# Check the trade direction: BUY (=1) or SELL (=0)
BUY = predictions[-1]

# Get the current price of BTC in USD
eth_usd_price = alpaca_api.get_latest_crypto_orderbook(['ETH/USD'])['ETH/USD'].asks[0].p

# Calculate the quantity of BTC to buy or sell
usd_balance = float(alpaca_api.get_account().cash)
quantity_to_trade = usd_balance / eth_usd_price * 0.1

# Define order parameters
symbol = 'ETHUSD'
order_type = 'limit'
limit_price = eth_usd_price  # limit price for the order
time_in_force = 'gtc'
qty = quantity_to_trade  # quantity of ETH to trade

if BUY:
    side = 'buy'
    limit_price += 0.0001
else:
    side = 'sell'
    qty = float(alpaca_api.get_position(symbol).qty)  # quantity of ETH to sell (same as the current position)
    limit_price -= 0.0001

# Place order
try:
    order = alpaca_api.submit_order(
        symbol=symbol,
        qty=qty,
        side=side,
        type=order_type,
        time_in_force=time_in_force,
        limit_price=limit_price
    )
    print(f"{side.capitalize()} order for {qty} {symbol} at {limit_price} submitted successfully.")

except Exception as e:
    print(f"Error submitting {side.capitalize()} order: ", e)



/Users/adriencaudron/miniforge3/envs/dev310/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 1.2.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Buy order for 3.989147426002818 ETHUSD at 1654.0001 submitted successfully.
